In [1]:
import pandas as pd
import numpy as np

from memory_profiler import profile

from Pyfhel import Pyfhel, PyPtxt, PyCtxt

import torch
import torch.nn as nn

import time
import os
import sys

working_directory = "/home/falcetta/PINPOINT_Secret"

device = "cpu"
module_path = os.path.abspath(working_directory)
sys.path.append(module_path) 

from pycrcnn.net_builder.encoded_net_builder_ts import build_from_pytorch
from pycrcnn.crypto.crypto import encrypt_matrix, decrypt_matrix
from train_utils import *

from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import mean_squared_error, mean_absolute_error

# Models

In [2]:
class Square(torch.nn.Module):
    def __init__(self):
        super().__init__()
 
    def forward(self, t):
        return torch.pow(t, 2)

class Cube(torch.nn.Module):
    def __init__(self):
        super().__init__()
 
    def forward(self, t):
        return torch.pow(t, 3)
    
class Printer(torch.nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, t):
        # print(t)
        print(t.shape)
        return t


class PINPOINT_1CONV(nn.Module):
    def __init__(self, input_size, output_horizon):
        super(PINPOINT_1CONV, self).__init__()

        n_kernels_1 = 32
        kernel_size_1 = 3
        out_conv_1 = n_kernels_1 * (input_size - kernel_size_1 + 1)

        self.main = nn.Sequential(           
            nn.Conv1d(in_channels=1, out_channels=n_kernels_1, kernel_size=kernel_size_1),
            Square(),
            nn.Flatten(),      
            
            nn.Linear(out_conv_1, int(out_conv_1/2)), #use without avgpool
            # nn.Linear(int(out_conv_1/2), output_horizon)   
            nn.Linear(int(out_conv_1/2), int(out_conv_1/4)),
            nn.Linear(int(out_conv_1/4), output_horizon)   
        )

    def forward(self, x):
        out = self.main(x)
        return out
    
    def __str__(self):
        return "PINPOINT_1CONV"

    
class PINPOINT_2CONV(nn.Module):
    def __init__(self, input_size, output_horizon):
        super(PINPOINT_2CONV, self).__init__()
        
        n_kernels_1 = 16
        n_kernels_2 = 32
        kernel_size_1 = 5
        kernel_size_2 = 3
        
        out_conv_1 = input_size - kernel_size_1 + 1
        out_conv_2 = n_kernels_2 * (out_conv_1 - kernel_size_2 + 1)

        self.main = nn.Sequential(           
            nn.Conv1d(in_channels=1, out_channels=n_kernels_1, kernel_size=kernel_size_1),
            Square(),
            nn.Conv1d(in_channels=n_kernels_1, out_channels=n_kernels_2, kernel_size=kernel_size_2),
            Square(),
            nn.Flatten(),      
            
            nn.Linear(out_conv_2, int(out_conv_2/2)), #use without avgpool
            # nn.Linear(int(out_conv_2/4), output_horizon)   
            nn.Linear(int(out_conv_2/2), int(out_conv_2/4)),
            nn.Linear(int(out_conv_2/4), output_horizon)   
        )

    def forward(self, x):
        out = self.main(x)
        return out
    
    def __str__(self):
        return "PINPOINT_2CONV"

In [3]:
experiment_name = "Milk"
seq_length = 12
forecast_horizon = 6
model_class = "PINPOINT_2CONV"

In [4]:
model = torch.load(f"{working_directory}/Experiments/models/{experiment_name}_{forecast_horizon}_{model_class}.pt")

In [5]:
model

PINPOINT_2CONV(
  (main): Sequential(
    (0): Conv1d(1, 16, kernel_size=(5,), stride=(1,))
    (1): Square()
    (2): Conv1d(16, 32, kernel_size=(3,), stride=(1,))
    (3): Square()
    (4): Flatten(start_dim=1, end_dim=-1)
    (5): Linear(in_features=192, out_features=96, bias=True)
    (6): Linear(in_features=96, out_features=48, bias=True)
    (7): Linear(in_features=48, out_features=6, bias=True)
  )
)

# Dataset

In [6]:
milk_production = milk_production = pd.read_csv(f"{working_directory}/data/monthly-milk-production.csv", parse_dates=["Month"], index_col="Month")
milk_production = milk_production.loc[:, 'Production']
milk_production.index.freq = 'MS'
entire_ts = milk_production
train = milk_production.loc[:pd.Timestamp("1974-01-01")]
validation_length = int(0.05 * len(train))
validation = entire_ts.loc[train.index[-1] + entire_ts.index.freq:train.index[-1] + validation_length * entire_ts.index.freq]
test = entire_ts.loc[validation.index[-1] + entire_ts.index.freq:]
plot_name = "Monthly milk production"
yaxis_name = "Production"

train = train.append(validation)

print(train)
print(test)

Month
1962-01-01    589
1962-02-01    561
1962-03-01    640
1962-04-01    656
1962-05-01    727
             ... 
1974-04-01    902
1974-05-01    969
1974-06-01    947
1974-07-01    908
1974-08-01    867
Freq: MS, Name: Production, Length: 152, dtype: int64
Month
1974-09-01    815
1974-10-01    812
1974-11-01    773
1974-12-01    813
1975-01-01    834
1975-02-01    782
1975-03-01    892
1975-04-01    903
1975-05-01    966
1975-06-01    937
1975-07-01    896
1975-08-01    858
1975-09-01    817
1975-10-01    827
1975-11-01    797
1975-12-01    843
Freq: MS, Name: Production, dtype: int64


# Expected outputs

In [7]:
expected_output = []

scaler = MinMaxScaler(feature_range=(-1, 1))
_ = scaler.fit_transform(train.values.reshape(-1, 1))

_train = train.copy()
_test = test.copy()

forecast = np.array([])

for i in range(0, int(len(_test) / forecast_horizon) + 1):
    model.eval()

    inputs = _train.values.reshape(len(_train), 1)

    inputs_normalized = scaler.transform(inputs)
    inputs_normalized = torch.FloatTensor(inputs_normalized[-seq_length:]).to(device)

    predict = model(inputs_normalized.reshape(1, 1, seq_length))
    predict = scaler.inverse_transform(predict.cpu().detach().numpy())
    forecast = np.append(forecast, predict)

    for j in range(0, forecast_horizon):
        if len(_test) > 0:
            _train[_train.index[-1] + train.index.freq] = _test.iloc[0]
            _test = _test.iloc[1:]

expected_output = pd.Series(data=forecast[:len(test)], index=test.index)

In [8]:
expected_output

Month
1974-09-01    812.044067
1974-10-01    823.361755
1974-11-01    764.408569
1974-12-01    813.593750
1975-01-01    839.368958
1975-02-01    790.813354
1975-03-01    882.921692
1975-04-01    899.379700
1975-05-01    968.319763
1975-06-01    939.873291
1975-07-01    895.119324
1975-08-01    852.359619
1975-09-01    811.907104
1975-10-01    820.040039
1975-11-01    766.692261
1975-12-01    808.899536
Freq: MS, dtype: float64

In [9]:
print(experiment_name)
print(f"MAE of model {model}, forecast horizon: {forecast_horizon}: {round(mean_absolute_error(test, expected_output), 2)}")

Milk
MAE of model PINPOINT_2CONV, forecast horizon: 6: 8.66


## Encode the models

In [10]:
HE = Pyfhel()    
HE.contextGen(p=96155351715128, m=8192, intDigits=16, fracDigits=64) 
HE.keyGen()
HE.relinKeyGen(30, 3)

encoded_model = build_from_pytorch(HE, model.cpu().main)

# Encrypted processing

In [11]:
decrypted_output = None

scaler = MinMaxScaler(feature_range=(-1, 1))
_ = scaler.fit_transform(train.values.reshape(-1, 1))

_train = train.copy()
_test = test.copy()

forecast = np.array([])

for i in range(0, int(len(_test) / forecast_horizon) + 1):
    inputs = _train.values.reshape(len(_train), 1)

    inputs_normalized = scaler.transform(inputs)
    inputs_normalized = inputs_normalized[-seq_length:].reshape(1, 1, seq_length)

    encrypted_input = encrypt_matrix(HE, inputs_normalized)

    for layer in encoded_model:
        encrypted_input = layer(encrypted_input)

    predict = decrypt_matrix(HE, encrypted_input)

    predict = scaler.inverse_transform(predict)
    forecast = np.append(forecast, predict)

    for j in range(0, forecast_horizon):
        if len(_test) > 0:
            _train[_train.index[-1] + train.index.freq] = _test.iloc[0]
            _test = _test.iloc[1:]

decrypted_output = pd.Series(data=forecast[:len(test)], index=test.index)

In [12]:
expected_output

Month
1974-09-01    812.044067
1974-10-01    823.361755
1974-11-01    764.408569
1974-12-01    813.593750
1975-01-01    839.368958
1975-02-01    790.813354
1975-03-01    882.921692
1975-04-01    899.379700
1975-05-01    968.319763
1975-06-01    939.873291
1975-07-01    895.119324
1975-08-01    852.359619
1975-09-01    811.907104
1975-10-01    820.040039
1975-11-01    766.692261
1975-12-01    808.899536
Freq: MS, dtype: float64

In [13]:
decrypted_output

Month
1974-09-01    812.070318
1974-10-01    823.409444
1974-11-01    764.445482
1974-12-01    813.697553
1975-01-01    839.457130
1975-02-01    790.983269
1975-03-01    882.942400
1975-04-01    899.404986
1975-05-01    968.421616
1975-06-01    939.935210
1975-07-01    895.238056
1975-08-01    852.496485
1975-09-01    811.963993
1975-10-01    820.129639
1975-11-01    766.801918
1975-12-01    809.057570
Freq: MS, dtype: float64

In [14]:
print(f"MAE of model {model}, forecast horizon: {forecast_horizon}: {round(mean_absolute_error(test, expected_output), 2)}")
print(f"MAE of model {model} used on encrypted inputs, forecast horizon: {forecast_horizon}: {round(mean_absolute_error(test, decrypted_output), 2)}")

MAE of model PINPOINT_2CONV, forecast horizon: 6: 8.66
MAE of model PINPOINT_2CONV used on encrypted inputs, forecast horizon: 6: 8.65


Difference between expected and obtained on encrypted data:

In [15]:
print(expected_output - decrypted_output)

Month
1974-09-01   -0.026250
1974-10-01   -0.047689
1974-11-01   -0.036912
1974-12-01   -0.103803
1975-01-01   -0.088173
1975-02-01   -0.169914
1975-03-01   -0.020708
1975-04-01   -0.025287
1975-05-01   -0.101852
1975-06-01   -0.061919
1975-07-01   -0.118732
1975-08-01   -0.136866
1975-09-01   -0.056888
1975-10-01   -0.089600
1975-11-01   -0.109657
1975-12-01   -0.158033
Freq: MS, dtype: float64
